In [46]:
import numpy as np
import cv2
np.random.seed(17785)

In [47]:
# read depth image
depth_scale = 0.00012498664727900177
depth_img = cv2.imread('depth.png')
dpt = depth_img[:, :, 2] + depth_img[:, :, 1] * 256
dpt = dpt * depth_scale

# read seg image
seg = cv2.imread('seg.png')[...,0]  # 255: fore ground, 0: background

# 看看 dpt (深度图) 和 seg (mask图) 长啥样
if show := False:
    import matplotlib.pyplot as plt
    plt.matshow(dpt); plt.show()
    plt.matshow(seg); plt.show()

# read intrinsics and extrinsics
K = np.load('intrinsic.npy')
print(K)

[[415.69219382   0.         320.        ]
 [  0.         415.69219382 240.        ]
 [  0.           0.           1.        ]]


In [48]:
# task1: convert depth image to point cloud
def depth2pc(depth, seg, intrinsic_mat):
    # ------------TODO---------------
    # compute point cloud from depth image
    # for-loop is not allowed!!
    # ------------TODO --------------

    # 找出 mask 对应的那些 (x, y, depth)
    y_grid, x_grid = np.meshgrid(range(depth.shape[1]), range(depth.shape[0]))
    mask = seg.astype(bool)
    x = x_grid[mask]
    y = y_grid[mask]
    x, y = y, x         # 这里是亡羊补牢的修正, 因为题目似乎认为 x 是水平的, y 是竖直的
    depth = depth[mask]
    # 补成齐次坐标
    xy1 = np.vstack([x, y, np.ones(x.shape)])
    # 计算三维坐标 (当然, 还差一个 [倍数])
    xyz = np.linalg.inv(intrinsic_mat) @ xy1
    # 下面根据深度值 (z 值!) 来计算这个 [倍数] 并应用到 xyz 上
    current_z = xyz[-1]
    target_z = depth
    ratio = target_z / current_z
    pc = (xyz * ratio).T.copy()
    return pc

partial_pc = depth2pc(dpt, seg, K)

# For debug and submission
np.savetxt('../results/pc_from_depth.txt', partial_pc)

In [49]:
# task2: compute one-way chamfer distance to the complete shape
# Note that 'full_pc.txt' is the point cloud sampled from the mesh before rendering to depth image.
# Since the default direction of the camera in PyRender is (0,0,-1), you may find the results of your back projection is not aligned with 'full_pc.txt'.
# To avoid confusion, we provide 'aligned_full_pc.txt', which should be aligned with your resulted point cloud.
full_pc = np.loadtxt('aligned_full_pc.txt')

def random_sample(pc, num):
    permu = np.random.permutation(pc.shape[0])
    return pc[permu][:num]

partial_pc_sampled = random_sample(partial_pc, 2048)
full_pc_sampled = random_sample(full_pc, 2048)

# -----------TODO---------------
# implement one way chamfer distance
# -----------TODO---------------

def one_way_chamfer_distance(approx: np.ndarray, target: np.ndarray):
    approx = approx.reshape((approx.shape[0], 1, 3))
    target = target.reshape((1, target.shape[0], 3))
    dist_mat = np.sqrt(np.sum((approx - target) ** 2, axis=-1))
    chf_dist = np.mean(np.min(dist_mat, axis=1))
    return chf_dist

one_way_CD = one_way_chamfer_distance(partial_pc_sampled, full_pc_sampled)
print('one way chamfer distance: ', one_way_CD)

# For submission
np.savetxt('../results/one_way_CD.txt', [one_way_CD])

one way chamfer distance:  0.00974205457691213
